In [2]:
import requests

In [3]:
model_list_url = 'http://localhost:11434/api/tags'
data = requests.get(model_list_url).json()

In [4]:
data

{'models': [{'name': 'gemma:2b-instruct-q8_0',
   'model': 'gemma:2b-instruct-q8_0',
   'modified_at': '2024-04-30T08:20:50.203921775Z',
   'size': 2669360976,
   'digest': '233b6c11a9a965cf7788e05250f1d652c8ca491ca51aac9a0c58a5e165553d23',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gemma',
    'families': ['gemma'],
    'parameter_size': '3B',
    'quantization_level': 'Q8_0'}},
  {'name': 'gemma:7b-instruct-q8_0',
   'model': 'gemma:7b-instruct-q8_0',
   'modified_at': '2024-04-30T06:41:25.186224708Z',
   'size': 9077854096,
   'digest': '61d0f0df36377df4f3605e26e1278212939a9efba3c800863d1cea767e3068e3',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gemma',
    'families': ['gemma'],
    'parameter_size': '9B',
    'quantization_level': 'Q8_0'}},
  {'name': 'llama3:latest',
   'model': 'llama3:latest',
   'modified_at': '2024-04-21T17:13:50.251857665Z',
   'size': 4661224596,
   'digest': '71a106a910165bc11d43aa128d99e32576a91cbd

In [5]:
def create_prompt_scope3(statement: str) -> str:
    
    prompt = ''' Determine whether the sentence is discussing scope 3 emissions or not.
    Scope 3 categories are:
    Purchased goods and services
    Capital goods
    Fuel and energy-related activities (not included in scope 1 or 2)
    Upstream transportation and distribution
    Waste generated in operations
    Business travel
    Employee commuting
    Upstream leased assets
    Downstream transportation and distribution
    Processing of sold products
    Use of sold products
    End-of-life treatment of sold products
    Downstream leased assets
    Franchises
    Investments
    
 if yes, return the category of scope 3. Take a deep breath and answer.
    {statement} Response:'''
    
    modified_prompt = prompt.format(statement = statement)
    
    return modified_prompt

In [6]:
def create_prompt_vagueness(statement: str) -> str:
    
    prompt = ''' You will be given sentences from corporate ESG reports. Your task is to classify each sentence into one of the following categories:

1. Specific: The sentence describes specific and measurable actions, initiatives, or achievements related to ESG.

2. Ambiguous: The sentence mentions ESG-related topics but lacks clear details or measurable commitments.

3. Generic: The sentence contains generic statements or platitudes about ESG without specific details.

4. NotRelated: The sentence is not related to ESG topics.

For each sentence, respond with the corresponding category number and a brief explanation for your classification.

Example:

Sentence: "Our company has implemented energy-efficient lighting across all facilities, reducing carbon emissions by 25% in 2022."
Response: 1. Specific - This sentence describes a concrete action (implementing energy-efficient lighting) and provides a measurable outcome (25% reduction in carbon emissions in 2022).

Sentence: "We are committed to environmental stewardship and sustainable practices."
Response: 3. Generic - This is a generic statement about environmental commitment without any specific details or actions.
    {statement} Response:'''
    
    modified_prompt = prompt.format(statement = statement)
    
    return modified_prompt

In [7]:
create_prompt_vagueness('The company has implemented energy-efficient lighting across all facilities, reducing carbon emissions by 25% in 2022.')

' You will be given sentences from corporate ESG reports. Your task is to classify each sentence into one of the following categories:\n\n1. Specific: The sentence describes specific and measurable actions, initiatives, or achievements related to ESG.\n\n2. Ambiguous: The sentence mentions ESG-related topics but lacks clear details or measurable commitments.\n\n3. Generic: The sentence contains generic statements or platitudes about ESG without specific details.\n\n4. Not Related: The sentence is not related to ESG topics.\n\nFor each sentence, respond with the corresponding category number and a brief explanation for your classification.\n\nExample:\n\nSentence: "Our company has implemented energy-efficient lighting across all facilities, reducing carbon emissions by 25% in 2022."\nResponse: 1. Specific - This sentence describes a concrete action (implementing energy-efficient lighting) and provides a measurable outcome (25% reduction in carbon emissions in 2022).\n\nSentence: "We are

In [8]:
model_chat_url = 'http://localhost:11434/api/chat'
input_sentence = '''Microsoft has made a commitment to become carbon negative by 2030, and to remove the impact of all
our historical emissions by 2050.1 Because of this commitment, we are compelled to focus on Scope 3
emissions, representing one of the largest contributors to greenhouse gas emissions, and attributed in
great measure to processes within our supply chains'''
prompt = create_prompt_vagueness(input_sentence)
json_data = {
    "model":"mistral:7b-instruct-v0.2-q8_0",
    "stream":False,
    "messages": [
        {
            "role":"user",
            "content": prompt
        }
    ],
        "options": {"num_predict":20, "temperature":0.2}
}
requests.post(model_chat_url, json=json_data).json()

{'model': 'mistral:7b-instruct-v0.2-q8_0',
 'created_at': '2024-04-30T09:52:26.856364376Z',
 'message': {'role': 'assistant',
  'content': " 1. Specific - Microsoft's commitment to becoming carbon negative by 2030 and"},
 'done': True,
 'total_duration': 7694234260,
 'load_duration': 7157267478,
 'prompt_eval_count': 350,
 'prompt_eval_duration': 175170000,
 'eval_count': 20,
 'eval_duration': 229917000}

In [28]:
def get_chat_response(text):
    model_chat_url = 'http://localhost:11434/api/chat'
    prompt = create_prompt_scope3(text)
    json_data = {
        "model":"mistral:7b-instruct-v0.2-q8_0",
        "stream":False,
        "messages": [
            {
                "role":"user",
                "content": prompt
            }
        ],
        "options": {"num_predict":20}
    }
    json_response_scope3 = requests.post(model_chat_url, json=json_data).json()['message']['content']

    prompt = create_prompt_vagueness(text)
    json_data = {
        "model":"mistral:7b-instruct-v0.2-q8_0",
        "stream":False,
        "messages": [
            {
                "role":"user",
                "content": prompt
            }
        ],
        "options": {"num_predict":20, "temperature":0.2}
    }
    json_response_vagueness = requests.post(model_chat_url, json=json_data).json()['message']['content']
    return json_response_scope3, json_response_vagueness

In [29]:
get_chat_response(input_sentence)

(' Yes, the sentence is discussing Scope 3 emissions. The category mentioned in the text is "',
 " 1. Specific - Microsoft's commitment to becoming carbon negative by 2030 and")

In [30]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

data = pd.read_csv('final_annotated_data.csv')
data['chat_responses'] = data['text'].progress_apply(get_chat_response)
data['chat_response_scope3'] = data['chat_responses'].apply(lambda x: x[0])
data['chat_response_vague'] = data['chat_responses'].apply(lambda x: x[1])

# data['scope3'] = data['chat_response'].apply(lambda x: x['scope3'])
# data['vague'] = data['chat_response'].apply(lambda x: x['vague'])


  0%|          | 0/10834 [00:00<?, ?it/s]

100%|██████████| 10834/10834 [2:19:08<00:00,  1.30it/s] 


In [31]:
data.head()

,text,gpt_responses,scope3,vague,file_name,chat_responses,chat_response_scope3,chat_response_vague
0,This report contains links to websites that ar...,"{'scope3': 'no', 'vague': 'notESG'}",no,notESG,NYSE_DIS_2022_results.csv,"( Based on the given sentence, it is not discu...","Based on the given sentence, it is not discus...",4. Not Related - This sentence does not relat...
1,The website owner’s Terms of Use and Privacy P...,"{'scope3': 'no', 'vague': 'notESG'}",no,notESG,NYSE_DIS_2022_results.csv,"( Based on the provided sentence, it does not ...","Based on the provided sentence, it does not a...",4. Not Related - These sentences are not rela...
2,"At The Walt Disney Company, our mission is to ...","{'scope3': 'no', 'vague': 'notESG'}",no,notESG,NYSE_DIS_2022_results.csv,( The sentence does not discuss Scope 3 emissi...,The sentence does not discuss Scope 3 emissio...,4. Not Related - This sentence does not menti...
3,"Since our founding nearly 100 years ago, story...","{'scope3': 'no', 'vague': 'notESG'}",no,notESG,NYSE_DIS_2022_results.csv,"( Based on the given sentence, there is no cle...","Based on the given sentence, there is no clea...",4. Not Related - This sentence does not menti...
4,This report shares our track record and ongoin...,"{'scope3': 'no', 'vague': 'generic'}",no,generic,NYSE_DIS_2022_results.csv,"( Based on the given sentence, it does not app...","Based on the given sentence, it does not appe...",4. Not Related - While ESG can include social...


In [32]:
data.to_csv('mistral7b_baseline_new.csv', index=False)

In [9]:
def get_chat_response(text):
    model_chat_url = 'http://localhost:11434/api/chat'
    prompt = create_prompt_scope3(text)
    json_data = {
        "model":"gemma:7b-instruct-q8_0",
        "stream":False,
        "messages": [
            {
                "role":"user",
                "content": prompt
            }
        ],
        "options": {"num_predict":20}
    }
    json_response_scope3 = requests.post(model_chat_url, json=json_data).json()['message']['content']

    prompt = create_prompt_vagueness(text)
    json_data = {
        "model":"gemma:7b-instruct-q8_0",
        "stream":False,
        "messages": [
            {
                "role":"user",
                "content": prompt
            }
        ],
        "options": {"num_predict":20, "temperature":0.2}
    }
    json_response_vagueness = requests.post(model_chat_url, json=json_data).json()['message']['content']
    return json_response_scope3, json_response_vagueness

In [11]:
get_chat_response(input_sentence)

('Yes, the sentence is discussing scope 3 emissions. The text specifically mentions "Scope 3 emissions',
 '## Sentence Classification\n\n**1. Specific:**\n\n**Sentence:** "Our company has implemented energy-')

In [12]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

data = pd.read_csv('final_annotated_data.csv')
data['chat_responses'] = data['text'].progress_apply(get_chat_response)
data['chat_response_scope3'] = data['chat_responses'].apply(lambda x: x[0])
data['chat_response_vague'] = data['chat_responses'].apply(lambda x: x[1])

# data['scope3'] = data['chat_response'].apply(lambda x: x['scope3'])
# data['vague'] = data['chat_response'].apply(lambda x: x['vague'])


100%|██████████| 10834/10834 [3:22:57<00:00,  1.12s/it] 


In [14]:
data.to_csv('gemma7b_baseline_new.csv', index=False)

In [15]:
!ls -lha

total 1.8G
drwxrwxr-x 6 siva siva 4.0K Apr 30 19:00  .
drwxrwxr-x 6 siva siva 4.0K Apr 21 19:20  ..
-rw-rw-r-- 1 siva siva 216K Apr 21 22:05  Alpaca_+_Mistral_7b_full_example.ipynb
-rw-rw-r-- 1 siva siva 2.1M Apr 29 12:28  baseline_data_mistral7b.csv
-rw-rw-r-- 1 siva siva 1.7G Apr 29 22:56  best_model.pth
-rw-r--r-- 1 siva siva  97K Apr 21 21:14  Copy_of_CS685_S24_HW1.ipynb
drwxrwxr-x 5 siva siva 4.0K Apr 30 08:51  CS685_final_project
-rw-rw-r-- 1 siva siva 135K Apr 25 02:00  DE_gpt_2.txt
-rw-rw-r-- 1 siva siva 163K Apr 25 09:29  DE_gpt3.txt
-rw-rw-r-- 1 siva siva 146K Apr 25 09:29  DE_gpt_4.txt
-rw-rw-r-- 1 siva siva 154K Apr 25 00:05  DE_gpt.txt
-rw-rw-r-- 1 siva siva  51K Apr 24 21:50  DE_llm.txt
-rw-rw-r-- 1 siva siva 123K Apr 27 16:39  DE_rules_cleaning.txt
-rw-rw-r-- 1 siva siva 180K Apr 24 17:12  DE_sustain_report_parsed.txt
-rw-rw-r-- 1 siva siva  14M Apr 25 01:29  DE_sustain_report.pdf
-rw-rw-r-- 1 siva siva 9.6K Apr 24 23:27  de_test.txt
-rw-rw-r-- 1 siva siva 122K Apr 30 18

In [27]:
def get_chat_response(text):
    model_chat_url = 'http://localhost:11434/api/chat'
    prompt = create_prompt_scope3(text)
    json_data = {
        "model":"gemma:2b-instruct-q8_0",
        "stream":False,
        "messages": [
            {
                "role":"user",
                "content": prompt
            }
        ],
        "options": {"num_predict":50}
    }
    json_response_scope3 = requests.post(model_chat_url, json=json_data).json()['message']['content']

    prompt = create_prompt_vagueness(text)
    json_data = {
        "model":"gemma:2b-instruct-q8_0",
        "stream":False,
        "messages": [
            {
                "role":"user",
                "content": prompt
            }
        ],
        "options": {"num_predict":50, "temperature":0.2}
    }
    json_response_vagueness = requests.post(model_chat_url, json=json_data).json()['message']['content']
    return json_response_scope3, json_response_vagueness

In [28]:
get_chat_response(input_sentence)

('The sentence is discussing scope 3 emissions, representing one of the largest contributors to greenhouse gas emissions, and attributed in great measure to processes within our supply chains.',
 "Sure, here's the classification of the sentences you provided:\n\n- Sentence 1: 1. Specific\n- Sentence 2: 3. Generic\n- Sentence 3: 2. Ambiguous\n- Sentence 4:")

In [24]:
from concurrent.futures import ThreadPoolExecutor
from concurrent import futures as f

In [33]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

data = pd.read_csv('final_annotated_data.csv')
data['chat_responses'] = data['text'].progress_apply(get_chat_response)
data['chat_response_scope3'] = data['chat_responses'].apply(lambda x: x[0])
data['chat_response_vague'] = data['chat_responses'].apply(lambda x: x[1])

# data['scope3'] = data['chat_response'].apply(lambda x: x['scope3'])
# data['vague'] = data['chat_response'].apply(lambda x: x['vague'])


100%|██████████| 10834/10834 [2:33:59<00:00,  1.17it/s] 


In [34]:
data.to_csv('gemma2b_baseline_new.csv', index=False)